<a href="https://colab.research.google.com/github/elichen/karpathyGPT/blob/main/Let's_build_GPT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [29]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [30]:
block_size = 8
batch_size = 32
learning_rate = 1e-3
max_inters = 3000
device = "cuda" if torch.cuda.is_available() else "cpu"
eval_iters = 200
eval_interval = 300

In [31]:
!wget -q https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

In [32]:
text = open("input.txt").read()

In [33]:
chars = sorted(list(set(text)))
vocab_size = len(chars)
vocab_size, "".join(chars)

(65, "\n !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz")

In [34]:
stoi = {ch:i for i,ch in enumerate(chars)}
itos = {i:ch for i,ch in enumerate(chars)}
encode = lambda s: [stoi[c] for c in s]
decode = lambda l: "".join([itos[i] for i in l])

In [35]:
data = torch.tensor(encode(text))
n = int(0.9*len(data))
train_data = data[:n]
val_data = data[n:]

In [36]:
def get_batch(split):
  data = train_data if split == 'train' else val_data
  ix = torch.randint(len(data) - block_size, (batch_size,))
  x = torch.stack([data[i:i+block_size] for i in ix])
  y = torch.stack([data[i+1:i+block_size+1] for i in ix])
  return x.to(device), y.to(device)

In [37]:
class BigramModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.embeddings = nn.Embedding(vocab_size, vocab_size)

    def forward(self, inputs):
        embeds = self.embeddings(inputs)
        return embeds

    def calculate_loss(self, inputs, targets):
        logits = self(inputs)
        return F.cross_entropy(logits.view(-1, vocab_size), targets.view(-1))

    def generate(self, input_batch, max_new_tokens):
        generated = input_batch
        for _ in range(max_new_tokens):
            logits = self(generated)
            last_logits = logits[:, -1, :]
            probabilities = F.softmax(last_logits, dim=-1)
            next_tokens = torch.multinomial(probabilities, 1).squeeze(1)
            generated = torch.cat((generated, next_tokens.unsqueeze(1)), dim=1)
        return generated

In [38]:
@torch.no_grad()
def estimate_loss(model):
  out = {}
  model.eval()
  for split in ["train", "val"]:
    losses = torch.zeros(eval_iters)
    for k in range(eval_iters):
      X, Y = get_batch(split)
      loss = model.calculate_loss(X, Y)
      losses[k] = loss.item()
    out[split] = losses.mean()
  model.train()
  return out

In [39]:
model = BigramModel(vocab_size).to(device)
xb,yb = get_batch("train")
optimizer = optim.AdamW(model.parameters(), lr=learning_rate)

for steps in range(max_inters):
  if steps % eval_interval == 0:
    losses = estimate_loss(model)
    print(f"step {steps}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

  xb,yb = get_batch("train")
  loss = model.calculate_loss(xb,yb)
  optimizer.zero_grad()
  loss.backward()
  optimizer.step()
print(loss.item())

step 0: train loss 4.7976, val loss 4.7951
step 300: train loss 4.4611, val loss 4.4518
step 600: train loss 4.1616, val loss 4.1597
step 900: train loss 3.9066, val loss 3.8994
step 1200: train loss 3.6692, val loss 3.6781
step 1500: train loss 3.4715, val loss 3.4811
step 1800: train loss 3.3010, val loss 3.3150
step 2100: train loss 3.1529, val loss 3.1635
step 2400: train loss 3.0368, val loss 3.0494
step 2700: train loss 2.9352, val loss 2.9473
2.8120272159576416


In [40]:
context = torch.zeros((1,1), dtype=torch.long, device=device)
print(decode(model.generate(context, max_new_tokens=400)[0].tolist()))


BEOnent tey!GjomPSI.k uAK.kn!zF-aq3ajS LDonv;kncstjo?
DZou?
Inth's, dbaifato fgn nder my &'-f s,P'd,
Ie?
RKPfleatom:
Nsoks ldi??
KnTS:RUSchora
HSBAEEANI'dchUS sofs tsUv,
LO,: maqfeld, lg.redushithB-cilcDe'e k; f;
WLUCHIFz&OVQg fby rrbof fWQ?HEDens heWh iMondofLatoveo t OifetdLI gfccQway;CELdofNGI won,D: bFRIVln.N oras isenYOKEE&xathar O
BPkerC wHant ahBut BOhajryoukl-R.hur:Wie fMbyx
NONGg:
SHad? t
